In [ ]:
import pickle
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def clean_text(text):
    lemmatizer = WordNetLemmatizer()
    text = text.lower()
    text = re.sub('[.,:;_%©?*,!@#$%^&()]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    d = {'1':'one','2':'two','3':'three','4':'four','5':'five','6':'six','7':'seven','8':'eight','9':'nine'}
    text = ''.join([d.get(word, word) for word in text])
    text = ' '.join(word for word in text.split() if len(word)>1)
    return text

vect_file = 'vectorizer.pkl'
vectorizer = pickle.load(open(vect_file, 'rb'))
df = pd.read_csv('new.csv')

data = pd.DataFrame(df['description'])
data['description'] = data.description.map(lambda x: clean_text(x))
tfidf_matrix = vectorizer.transform(data['description'])

model_file = 'model.pkl'
model = pickle.load(open(model_file, 'rb'))
preds = model.predict(tfidf_matrix.toarray())
df['target'] = preds

In [ ]:
print(f'Founded {df.shape[0]} new offers')
for i, row in df.iterrows():
    print(row.url)
    print('Target = ', row.target)
    print('Is it correct classification?')
    ans = input('y/n')
    if ans == 'n':
        if df.target.loc[i] == 0:
            df.target.loc[i] = 1
        else:
            df.target.loc[i] = 0

In [ ]:
data_bank = pd.read_csv('data_bank.csv')
data_bank = pd.concat([data_bank,df],axis=0)
data_bank.drop_duplicates(inplace=True)
data_bank.to_csv('data_bank.csv',index=False)